<a href="https://colab.research.google.com/github/carlosdgerez/machine_learning/blob/main/daysResponses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# Importing the 2 sets. one for the model, and the second for final testing. 

import pandas as pd
import altair as alt

campaign = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/bank.csv')
last_campaign = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/bank_holdout_test.csv')

In [3]:
# slice of responses for marketing the customer was contacted at least once
contacted = campaign[campaign.pdays != 999]
contacted.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1515 entries, 24108 to 41182
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             1515 non-null   int64  
 1   job             1515 non-null   object 
 2   marital         1515 non-null   object 
 3   education       1515 non-null   object 
 4   default         1515 non-null   object 
 5   housing         1515 non-null   object 
 6   loan            1515 non-null   object 
 7   contact         1515 non-null   object 
 8   month           1515 non-null   object 
 9   day_of_week     1515 non-null   object 
 10  campaign        1515 non-null   int64  
 11  pdays           1515 non-null   int64  
 12  previous        1515 non-null   int64  
 13  poutcome        1515 non-null   object 
 14  emp.var.rate    1515 non-null   float64
 15  cons.price.idx  1515 non-null   float64
 16  cons.conf.idx   1515 non-null   float64
 17  euribor3m       1515 non-nul

In [4]:
# regroup in a new dataframe including the job column orderer by responses yes or no
# This will be the base for the comparative chart
contacted_regroup = pd.DataFrame(contacted.groupby(['month', 'day_of_week','job', 'y']).size()).reset_index()
contacted_regroup.rename(columns = {0:'totals'}, inplace= True)


In [16]:
contacted_regroup_yes = contacted_regroup[contacted_regroup['y'] == 'yes']

In [27]:
contacted_regroup

,month,day_of_week,job,y,totals
0,apr,fri,admin.,no,2
1,apr,fri,blue-collar,no,1
2,apr,fri,blue-collar,yes,1
3,apr,fri,retired,no,1
4,apr,fri,retired,yes,1
...,...,...,...,...,...
599,sep,wed,technician,yes,2
600,sep,wed,unemployed,no,1
601,sep,wed,unemployed,yes,2
602,sep,wed,unknown,no,1


In [12]:
# Lists to order output
week_days_orderer = ['mon','tue', 'wed','thu','fri']
month_names_in_order =['jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec']


In [36]:
# final interactive approach to show the interaction between month, days and jobs responses positives
brush = alt.selection_interval()

points = alt.Chart(contacted_regroup).mark_point().encode(
    x= alt.X('month', sort = month_names_in_order, axis =alt.Axis(labelAngle= 0)),
    y= alt.Y('day_of_week', sort = week_days_orderer),
    size= alt.Size('totals',legend = None),
    color=alt.condition(brush, 'job', alt.value('lightgray'))
).add_selection(
    brush
).properties(
    width = 1000,
    height = 300,
    title= 'Select a period.'
)

bars = alt.Chart(contacted_regroup).mark_bar().encode(
    y='job',
    color='job',
    x='sum(totals)'
).transform_filter(
    brush
).properties(
    title = ' Total number of  responses in period'
)

bars_yes = alt.Chart (contacted_regroup_yes).mark_bar().encode(
    y ='job',
    color = 'job',
    x= 'sum(totals)'
).transform_filter(
    brush
).properties(
    title=  'Yes responses in period by job'
)
(bars | bars_yes) & points

alt.VConcatChart(...)